In [1]:
#
# Read files
#

import pandas as pd
import numpy as np
import glob

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.style(theme='grade3')

In [2]:
allFiles = glob.glob("../data_test/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    dfTmp = pd.read_csv(file_,index_col=None, header=0, encoding="ISO-8859-1", thousands=',')
    dfTmp = dfTmp[['BIOGUIDE_ID', 'OFFICE', 'CATEGORY', 'START DATE', 'END DATE', 'PURPOSE', 'PAYEE', 'YEAR', 'AMOUNT']]
    list_.append(dfTmp)
df = pd.concat(list_)
df.info()

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542048 entries, 0 to 104104
Data columns (total 9 columns):
BIOGUIDE_ID    object
OFFICE         object
CATEGORY       object
START DATE     object
END DATE       object
PURPOSE        object
PAYEE          object
YEAR           object
AMOUNT         float64
dtypes: float64(1), object(8)
memory usage: 270.2+ MB


In [3]:
#
# Question 1
#
print("Question 1: {:.10}".format(df['AMOUNT'].sum()))

Question 1: 1.366070379e+10


In [4]:
#
# Question 2
#
df['START DATE'] = pd.to_datetime(df['START DATE'], format='%m/%d/%y', errors='coerce')
df['END DATE'] = pd.to_datetime(df['END DATE'], format='%m/%d/%y', errors='coerce')
df['COVERAGE PERIOD'] = (df['END DATE'] - df['START DATE'])
print("Question 2: {:.10}".format(df[df['AMOUNT']>0]['COVERAGE PERIOD'].std().total_seconds()/24/60/60))

Question 2: 61.88080826


In [5]:
#
# Question 3
#
years = range(2010,2017)
total = 0
for year in years:
    total += df[(df['START DATE'].dt.year==year) & (df['AMOUNT']>0)]['AMOUNT'].sum()
print("Question 3: {:.10}".format(total/len(years)))

Question 3: 1.209984621e+09


In [6]:
#
# Question 4
#
top_office = df[df['START DATE'].dt.year==2016][['OFFICE','AMOUNT']].groupby(['OFFICE'], as_index=False).sum().sort_values(by='AMOUNT', ascending=False).iloc[0]['OFFICE']
part_total = df[(df['START DATE'].dt.year==2016) & (df['OFFICE']==top_office)][['PURPOSE','AMOUNT']].groupby(['PURPOSE'], as_index=False).sum().sort_values(by='AMOUNT', ascending=False).iloc[0]['AMOUNT']
total = df[df['START DATE'].dt.year==2016]['AMOUNT'].sum()
print("Question 4: {:.10}".format(part_total/total))

Question 4: 0.05687370793


In [7]:
#
# Question 5
#
df['YEAR'] = df['YEAR'].str.extract('(\d\d\d\d)').astype('float')
df['BIOGUIDE_ID'].fillna('UNKNOWN', inplace=True)
df_OFFICE_PAYEE = df[(df['CATEGORY']=='PERSONNEL COMPENSATION') & (df['YEAR']==2016)][['BIOGUIDE_ID', 'PAYEE', 'AMOUNT']].groupby(['BIOGUIDE_ID','PAYEE'], as_index=False).sum()
print("Question 5: {:.10}".format(df_OFFICE_PAYEE[['BIOGUIDE_ID', 'AMOUNT']].groupby(['BIOGUIDE_ID'], as_index=False).mean().sort_values(by='AMOUNT', ascending=False).iloc[0]['AMOUNT']))

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


Question 5: 73816.42596


In [8]:
"""
import json
import requests

dictServeDate = {}
for member in df['BIOGUIDE_ID'].unique():
    url = "https://api.propublica.org/congress/v1/members/{}.json".format(member)
    response = requests.get(url, headers={'X-API-Key': 'X39vBTJI1ddXrbx3SCqNh4OzY70BCp6dwyAqUAxU'})
    minDate = pd.to_datetime('today')
    try:
        for role in json.loads(response.text)['results'][0]['roles']:
            if role['start_date'] and pd.to_datetime(role['start_date']) < minDate:
                minDate = pd.to_datetime(role['start_date'])
    except KeyError:
        pass
    dictServeDate[member] = minDate
"""

'\nimport json\nimport requests\n\ndictServeDate = {}\nfor member in df[\'BIOGUIDE_ID\'].unique():\n    url = "https://api.propublica.org/congress/v1/members/{}.json".format(member)\n    response = requests.get(url, headers={\'X-API-Key\': \'X39vBTJI1ddXrbx3SCqNh4OzY70BCp6dwyAqUAxU\'})\n    minDate = pd.to_datetime(\'today\')\n    try:\n        for role in json.loads(response.text)[\'results\'][0][\'roles\']:\n            if role[\'start_date\'] and pd.to_datetime(role[\'start_date\']) < minDate:\n                minDate = pd.to_datetime(role[\'start_date\'])\n    except KeyError:\n        pass\n    dictServeDate[member] = minDate\n'

In [9]:
"""
dictServe4 = set()
for member, date in dictServeDate.items():
    if (pd.to_datetime('today') - date).days >= 4*365:
        dictServe4.add(member)
"""

"\ndictServe4 = set()\nfor member, date in dictServeDate.items():\n    if (pd.to_datetime('today') - date).days >= 4*365:\n        dictServe4.add(member)\n"

In [10]:
#
# Question 6
#
dfTmp = df[['BIOGUIDE_ID', 'YEAR', 'PAYEE']].groupby(['BIOGUIDE_ID', 'YEAR'])['PAYEE'].nunique().reset_index()
dfTmp = dfTmp[dfTmp['PAYEE']>=5]
slcRep = set(dfTmp[['BIOGUIDE_ID', 'YEAR']].groupby(['BIOGUIDE_ID'], as_index=False).count()[dfTmp[['BIOGUIDE_ID', 'YEAR']].groupby(['BIOGUIDE_ID'], as_index=False).count()['YEAR']>=4]['BIOGUIDE_ID'])
df_turnover = df[df['BIOGUIDE_ID'].isin(slcRep)][['BIOGUIDE_ID', 'YEAR', 'PAYEE', 'AMOUNT']].groupby(['BIOGUIDE_ID', 'YEAR', 'PAYEE']).sum().reset_index()[['BIOGUIDE_ID', 'YEAR', 'PAYEE']]

In [11]:
df_turnover1 = df_turnover[['BIOGUIDE_ID', 'YEAR', 'PAYEE']].rename(index=str, columns={"YEAR": "NEXT YEAR"})
df_cmb = df_turnover.merge(df_turnover1, on=['BIOGUIDE_ID', 'PAYEE'], how='left')
df_cmb = df_cmb[(df_cmb['YEAR']==df_cmb['NEXT YEAR']) | (df_cmb['YEAR']+1==df_cmb['NEXT YEAR'])]
df_cmb["TURN OVER"] = df_cmb['YEAR']==df_cmb['NEXT YEAR']

In [12]:
years = range(2011,2017)
annualTurnOver = []
for year in years:
    dfTmp = df_cmb[df_cmb['YEAR']==year-1][['BIOGUIDE_ID', 'TURN OVER']].groupby(['BIOGUIDE_ID'], as_index=False).mean()
    annualTurnOver.append(dfTmp['TURN OVER'].mean())
print("Question 6: {:.10}".format(np.median(annualTurnOver)))

Question 6: 0.6488385221


In [13]:
#
# Question 7
#

import json
import requests

dictParty = {}
for member in df['BIOGUIDE_ID'].unique():
    url = "https://api.propublica.org/congress/v1/members/{}.json".format(member)
    response = requests.get(url, headers={'X-API-Key': 'X39vBTJI1ddXrbx3SCqNh4OzY70BCp6dwyAqUAxU'})
    try:
        dictParty[member] = json.loads(response.text)['results'][0]['current_party']
    except KeyError:
        dictParty[member] = 'NA'
        
top20 = df[(df['START DATE'].dt.year==2016) & (df['BIOGUIDE_ID']!='UNKNOWN')][['BIOGUIDE_ID', 'AMOUNT']].groupby(['BIOGUIDE_ID'], as_index=False).sum().sort_values(by='AMOUNT', ascending=False).iloc[:20][['BIOGUIDE_ID', 'AMOUNT']].values
democ, total = 0, 0
for member in top20:
    if dictParty[member[0]] == 'D':
        democ += member[1]
    total += member[1]
print("Question 7: {:.10}".format(democ/total))

Question 7: 0.4830437272
